In [26]:
import pandas as pd
import pyarrow.parquet as pq

In [27]:
DATA_DIR = '../data/'
anonym_http = pq.read_table(DATA_DIR + 'anonymousHTTPRequests-20180217.parquet').to_pandas()
bots = pq.read_table(DATA_DIR + 'botsUidBidTable.parquet').to_pandas()

In [28]:
anonym_http = anonym_http.sample(frac=0.003)

In [29]:
anonym_http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId
140962,1.847157e+18,None,UN4KSZub9Z/20BFMFx1z4g==,POST,videoStatNew,https://ok.ru/,REQ,1518768614925,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,null
241412,NaN,None,aSOUjJ21zcb40ucUGKicwg==,None,None,None,UPGRADE,1518746018317,None,http://ok.ru/videoembed/199337183932,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,null
412214,-6.581258e+18,None,2ez+D2rc8XDri06DWD6zpQ==,GET,WidgetGroup,https://nsportal.ru/shkola/biologiya/library/2...,NAV,1518783120397,WidgetGroup,https://connect.ok.ru/dk,Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_3 like...,null
875773,3.265704e+18,None,bLsUSGpXY7LnnndT8YvDGA==,GET,spring-mvc,https://ok.ru/,REQ,1518740726797,None,https://ok.ru/web-api/pts/video.player,Mozilla/5.0 (Windows NT 6.1; rv:51.0) Gecko/20...,null
338907,-1.480562e+18,None,oWZXIWJamReTA8pg2lV8eg==,GET,anonymFriendRestricted,https://ok.ru,NAV,1518799462925,anonymFriendRestricted,https://ok.ru/profile/515102403488,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2...,null


In [30]:
bots.head()
bots.shape

(1471, 2)

In [31]:
anonym_http.shape

(3478, 12)

In [32]:
# anonym_http = anonym_http.sample(frac=0.03)

In [33]:
anonym_http['bot_or_user'] = anonym_http['browserId'].apply(lambda x: 1 if x in bots['browserId'].values else 0)

In [34]:
len(anonym_http[anonym_http['bot_or_user'] == 1])

0

### Сделаем sparse matrix 

In [35]:
print('имеем {} уникальных операций'.format(anonym_http['operation'].unique().shape[0]))

имеем 68 уникальных операций


In [36]:
# закодируем операции интами
dict_op = {}

counter = 0
for op in anonym_http['operation'].unique():
    dict_op[op] = counter
    counter += 1

In [37]:
anonym_http['requestType'].unique()

array(['REQ', 'UPGRADE', 'NAV', 'CSP'], dtype=object)

In [38]:
anonym_http['requestType'].value_counts()

REQ        1830
NAV        1193
UPGRADE     421
CSP          34
Name: requestType, dtype: int64

In [39]:
# запилю сразу requestType
# dict_req = {}

# c = 0
# for rt in anonym_http['requestType']:
#     dict_req[rt] = c
#     c += 1
    
# anonym_http['requestType'] = anonym_http['requestType'].apply(lambda x: dict_req[x])    

In [40]:
anonym_http['operation'] = anonym_http['operation'].apply(lambda x: dict_op[x]);

In [41]:
from sklearn.preprocessing import OneHotEncoder

In [42]:
enc = OneHotEncoder(sparse=True)

In [43]:
encoded_categorical_columns = pd.DataFrame(enc.fit_transform(anonym_http[['operation']]).toarray())
encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
type(encoded_categorical_columns)

pandas.core.frame.DataFrame

In [45]:
anonym_http.assign(**encoded_categorical_columns)

TypeError: assign() keywords must be strings

In [22]:
anonym_http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,bot_or_user
348248,9.202510e+18,None,gP1faoushrJQujT2iKlyyA==,GET,0.0,None,NAV,1.518797e+12,anonymFriendAltGroup,https://ok.ru/profile/584587738157/groups,Apache-HttpAsyncClient/4.0.2 (java 1.5),null,0.0
74200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
